# Modulation and Coding Performance

SpaceLink includes a registry of modulation and coding combinations. Modes and the
associated error rate performance data is stored in a set of YAML files with a simple
schema such that new modes can be easily added.


In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

import astropy.units as u

from spacelink.phy.performance import ErrorMetric
from spacelink.phy.registry import Registry

## Usage Examples

The following examples demonstrate how to use the `spacelink.phy` module API to access mode properties, calculate bit rates, query performance data, and analyze link budgets.

As a first step we will load the registry from the YAML files in phy/data.

In [ ]:
registry = Registry()
registry.load()

### 1. Mode Properties

Each mode has properties that are relevant to link analysis. In this example a DVB-S2
mode with LDPC code rate 3/4 is inspected.


In [ ]:
# Access a specific mode
mode = registry.modes["DVBS2_QPSK_3/4"]

print(f"Mode ID: {mode.id}")
print(f"Modulation: {mode.modulation.name}")
print(f"Bits per symbol: {mode.modulation.bits_per_symbol}")
print(f"Code rate (includes PHY framing overhead): {mode.coding.rate}")
print(f"Information bits per symbol: {mode.info_bits_per_symbol}")
print(f"Channel bits per symbol: {mode.channel_bits_per_symbol}")
print(f"\nCoding chain ({len(mode.coding.codes)} stages):")
for i, code in enumerate(mode.coding.codes, 1):
    print(f"  {i}. {code.name} (rate: {code.rate})")

### 2. Bit Rate / Symbol Rate Conversions

Convert between symbol rates and information bit rates.


In [ ]:
# Example: Calculate information bit rate for a given symbol rate
symbol_rate = 1.0 * u.MHz
mode = registry.modes["DVBS2_16APSK_3/4"]

info_bit_rate = mode.info_bit_rate(symbol_rate)
print(f"Mode: {mode.id}")
print(f"Symbol rate: {symbol_rate}")
print(f"Information bit rate: {info_bit_rate.to_value(u.MHz):.2f} Mbps")

# Reverse calculation: Find required symbol rate for desired info bit rate
desired_bit_rate = 2.0 * u.MHz  # Actually Mbps, but we need to use Astropy units
required_symbol_rate = mode.symbol_rate(desired_bit_rate)
print(f"\nDesired information bit rate: {desired_bit_rate.value} Mbps")
print(f"Required symbol rate: {required_symbol_rate.to(u.kHz):.2f}")

### 3. Error Rate Performance Curves

Query performance curves to find error rates at specific $E_b/N_0$ values or determine required $E_b/N_0$ for target error rates.


In [ ]:
# Get a performance curve (CCSDS Rate 1/2 Turbo code)
curve = registry.get_performance_curve("CCSDS_TM_TURBO1784-12_BPSK", ErrorMetric.BER)

# Find error rate at specific Eb/N0
ebn0 = 1.2 * u.dB(1)
error_rate = curve.ebn0_to_error_rate(ebn0)
print(f"At Eb/N0 = {ebn0}:")
print(f"  Bit Error Rate: {error_rate:.4e}")

# Find required Eb/N0 for target error rate
target_ber = 1e-5 * u.dimensionless
required_ebn0 = curve.error_rate_to_ebn0(target_ber)
print(f"\nTo achieve BER = {target_ber}:")
print(f"  Required Eb/N0: {required_ebn0:.2f}")

# Array inputs are also supported
ebn0_range = np.array([0.8, 1.0, 1.2, 1.4]) * u.dB(1)
ber_values = curve.ebn0_to_error_rate(ebn0_range)
print("\nArray interpolation:")
for eb, ber in zip(ebn0_range, ber_values, strict=False):
    print(f"  Eb/N0 = {eb:.1f}, BER = {ber:.4e}")

### 4. Performance Threshold Usage

For some modulation and coding schemes full error rate curves are not published with the
standard, but instead threshold $E_b/N_0$ values are provided corresponding to some
defined (usually low) error rate. Sometimes this is named the "quasi-error-free" 
operating point since errors are so rare that they have no practical impact. SpaceLink
has special handling for this situation.


In [ ]:
# Get a DVB-S2 threshold for QPSK with code rate 3/4.
threshold = registry.get_performance_threshold("DVBS2_QPSK_3/4", ErrorMetric.PER)

print("Mode: DVBS2_QPSK_3/4")
print(
    f"Threshold: {threshold.ebn0:.2f} @ Packet Error Rate = {threshold.error_rate:.2e}"
)

# Check if link Eb/N0 meets the threshold
link_ebn0 = 3.5 * u.dB(1)
meets_threshold = threshold.check(link_ebn0)
margin = threshold.margin(link_ebn0)

print(f"\nLink Eb/N0: {link_ebn0}")
print(f"Meets quasi-error-free threshold: {meets_threshold}")
print(f"Link margin: {margin:.2f}")

# Check multiple scenarios
scenarios = np.array([1.0, 2.0, 3.0, 4.0]) * u.dB(1)
results = threshold.check(scenarios)
margins = threshold.margin(scenarios)

print("\nScenario analysis:")
for ebn0, meets, marg in zip(scenarios, results, margins, strict=False):
    status = "PASS" if meets else "FAIL"
    print(f"  Eb/N0 = {ebn0:.1f}: {status} (margin: {marg:+.2f})")

### 5. Coding Gain Calculation

Calculate the benefit of forward error correction relative to uncoded transmission.


In [ ]:
# Compare coded vs. uncoded performance
uncoded_curve = registry.get_performance_curve("UNCODED_BPSK", ErrorMetric.BER)
turbo_curve = registry.get_performance_curve(
    "CCSDS_TM_TURBO1784-12_BPSK", ErrorMetric.BER
)

# Calculate coding gain at specific BER
target_ber = 1e-5 * u.dimensionless
coding_gain = turbo_curve.coding_gain(uncoded_curve, target_ber)

uncoded_ebn0 = uncoded_curve.error_rate_to_ebn0(target_ber)
coded_ebn0 = turbo_curve.error_rate_to_ebn0(target_ber)

print(f"Coding gain analysis at BER = {target_ber}:")
print(f"  Uncoded BPSK requires: {uncoded_ebn0:.2f}")
print(f"  Turbo coded requires: {coded_ebn0:.2f}")
print(f"  Coding gain: {coding_gain:.2f}")

# Show coding gain across multiple BER values
ber_targets = np.array([1e-3, 1e-4, 1e-5]) * u.dimensionless
gains = turbo_curve.coding_gain(uncoded_curve, ber_targets)

print("\nCoding gain vs. BER:")
for ber, gain in zip(ber_targets, gains, strict=False):
    print(f"  BER = {ber:.1e}: {gain:.2f} coding gain")

---

## Interactive Performance Plot

The error rate performance of different modulation and coding options can be compared
in this interactive plot.


In [ ]:
modcod_registry = Registry()
modcod_registry.load()

mode_ids = sorted(modcod_registry.modes.keys())

ebn0 = np.linspace(-2, 10, 100) * u.dB(1)
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
fig.subplots_adjust(right=0.45)

# Define line styles for each error metric
error_metric_styles = {
    ErrorMetric.BER: {"linestyle": "-", "label_suffix": " (BER)"},
    ErrorMetric.WER: {"linestyle": "--", "label_suffix": " (WER)"},
    ErrorMetric.FER: {"linestyle": "-.", "label_suffix": " (FER)"},
    ErrorMetric.PER: {"linestyle": ":", "label_suffix": " (PER)"},
}


@widgets.interact(
    modcod_ids=widgets.SelectMultiple(
        options=mode_ids,
        value=(mode_ids[0],),
        description="Modes:",
        rows=min(12, len(mode_ids)),
        layout=widgets.Layout(width="50%", height="150px"),
        style={"description_width": "auto"},
    ),
    error_types=widgets.SelectMultiple(
        options=[
            ("Bit Error Rate (BER)", ErrorMetric.BER),
            ("Codeword Error Rate (WER)", ErrorMetric.WER),
            ("Frame Error Rate (FER)", ErrorMetric.FER),
            ("Packet Error Rate (PER)", ErrorMetric.PER),
        ],
        value=(ErrorMetric.BER, ErrorMetric.WER),
        description="Error Types:",
        rows=4,
        layout=widgets.Layout(width="50%", height="100px"),
        style={"description_width": "auto"},
    ),
)
def plot_modcods(modcod_ids, error_types):
    ax.clear()

    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

    # Track legend entries
    legend_handles = []
    legend_labels = []

    for idx, mode_id in enumerate(modcod_ids):
        color = colors[idx % len(colors)]

        for error_metric in error_types:
            # Try curve first, then threshold
            try:
                curve = modcod_registry.get_performance_curve(mode_id, error_metric)
                error_rate = curve.ebn0_to_error_rate(ebn0)

                style = error_metric_styles[error_metric]
                line_label = mode_id + style["label_suffix"]

                (line,) = ax.semilogy(
                    ebn0,
                    error_rate,
                    label=line_label,
                    color=color,
                    linestyle=style["linestyle"],
                )
                legend_handles.append(line)
                legend_labels.append(line_label)

            except KeyError:
                # Try threshold data
                try:
                    threshold = modcod_registry.get_performance_threshold(
                        mode_id, error_metric
                    )

                    style = error_metric_styles[error_metric]
                    scatter_label = mode_id + style["label_suffix"]

                    # Plot as scatter point with marker
                    scatter = ax.scatter(
                        [threshold.ebn0.value],
                        [threshold.error_rate.value],
                        marker="o",
                        color=color,
                        label=scatter_label,
                        zorder=10,
                    )
                    legend_handles.append(scatter)
                    legend_labels.append(scatter_label)

                except KeyError:
                    # No performance data for this mode/metric combination
                    continue

    ax.set_xlabel("$E_b/N_0$ (dB)")
    ax.set_ylabel("Error Rate")
    ax.grid(True)

    if legend_handles:
        ax.legend(
            legend_handles,
            legend_labels,
            loc="upper left",
            bbox_to_anchor=(1.02, 1.0),
            borderaxespad=0.0,
            title="Modes & Metrics",
            frameon=True,
        )